In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as ply
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from sklearn.model_selection import GridSearchCV

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
X = pd.read_csv('../../data/dengue_features_train.csv')
y = pd.read_csv('../../data/dengue_labels_train.csv', usecols=['total_cases'])

## Data Set Nomenclature
Train<br>
Validation - split off from training data to validate model<br>
Test - new data - the data that is used for competition submission<br>

In [3]:
X_test = pd.read_csv('../../data/dengue_features_test.csv')
y_test = pd.read_csv('../../data/submission_format.csv')

In [4]:
def data_preprocess(df):
    # drop or encode categorical cols
    df_processed = df.drop('week_start_date', axis=1)
    df_processed['city'] = df_processed['city'].apply(lambda x : 1 if x=='iq' else 0)
    return df_processed

In [25]:
def cross_validate(X, y, estimator, cv, scaler=StandardScaler(), imputer=KNNImputer(), dim_reduction=PCA()):
    pipeline = Pipeline(steps=[
        ('scaler', scaler),
        ('imputer', imputer),
        ('dim_reduction', dim_reduction),
        ('estimator', estimator)
    ])
    
    
    #X_train, y_train, X_val, y_val = train_test_split(X, y, test_size=.2, random_state=42)
    
    for train_idxs, val_idxs in cv.split(X, y):
        X_train, y_train = X.iloc[train_idxs], y.iloc[train_idxs]
        pipeline.fit(X_train, y_train)
        y_pred_train = pipeline.predict(X_train)
        print(f'Train MAE = {mean_absolute_error(y_train, y_pred_train)}')
        
        X_val, y_val = X.iloc[val_idxs], y.iloc[val_idxs]
        y_pred_val = pipeline.predict(X_val)

        print(f'Validation MAE = {mean_absolute_error(y_val, y_pred_val)}')


        return pipeline        

In [26]:
X_processed = data_preprocess(X)

estimator = RandomForestRegressor(n_estimators=100)
cv = KFold()
pipe = cross_validate(X_processed, y, estimator, cv)

/home/cgridley/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Train MAE = 4.615360824742268
Validation MAE = 39.88260273972603
